<a href="https://colab.research.google.com/github/atuch17/SyneraTech/blob/main/3_Neo4j.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# @title NEO4J CARGA DE DATOS
!pip install neo4j

from neo4j import GraphDatabase
import csv

path ='/content/drive/Shared drives/TFM/BBDD/EXPORTED/datos_exported/'

# Conectar a Neo4j Aura
uri = "neo4j+s://31a6faa8.databases.neo4j.io"
username = "neo4j"
password = "TztbsyPHVS41Nk6Uz2WLneL16u3mIreEMBjDsRpi3kw"

class Neo4jImporter:
    def __init__(self, uri, username, password):
        self._uri = uri
        self._username = username
        self._password = password
        self._driver = GraphDatabase.driver(uri, auth=(username, password))

    def import_csv(self, file_path, label):
        # Obtener el número total de filas en el archivo CSV
        with open(file_path, 'r', encoding='utf-8') as file:
            total_iterations = sum(1 for _ in file)

        # Leer datos del archivo CSV
        with open(file_path, 'r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            print('Abriendo fichero...')
            with self._driver.session() as session:
                for row in reader:
                    # Crear nodo para cada fila en el CSV
                    query = f"CREATE (n:{label} $props)"
                    params = {"props": row}
                    session.run(query, props=row)  # Corrección aquí

    def close(self):
        self._driver.close()


# Crear una instancia de Neo4jImporter
neo4j_importer = Neo4jImporter(uri, username, password)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.3/192.3 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j: filename=neo4j-5.13.0-py3-none-any.whl size=265313 sha256=aeaba32b65d3d33ba10e8b4afa4140689238478a658e3c62a499ce427715d994
  Stored in directory: /root/.cache/pip/wheels/7b/1d/b6/1be3a1e9de57bc832b7fcebbbf884186d8155bb6f1cc45be99
Successfully built neo4j


In [ ]:
# Subir nodos a Neo4j
neo4j_importer.import_csv(path + 'df_fusion.csv', 'Fus_Tot')
neo4j_importer.import_csv(path + 'df_fusion_pr.csv', 'Fus_Pr')
neo4j_importer.import_csv(path + 'df_fusion_dni.csv', 'Fus_Dni')
neo4j_importer.import_csv(path + 'df_fusion_ct_1.csv', 'Fus_Pr_CT')
neo4j_importer.import_csv(path + 'df_fusion_ct_2.csv', 'Fus_CT')
neo4j_importer.import_csv(path + 'df_fusion_rtc_3.csv', 'Fus_RT')
print('FIN')

Abriendo fichero...
Abriendo fichero...
Abriendo fichero...
Abriendo fichero...
Abriendo fichero...
Abriendo fichero...
FIN


In [ ]:
# @title Querys
neo4j_importer = Neo4jImporter(uri, username, password)
# ---------------------------------------------------------------------------------------------
with neo4j_importer._driver.session() as session:
  query_1_rel = """
  MATCH (p:Fus_Pr),(f:Fus_Tot), (u:Fus_Dni)
  WHERE p.project_id = f.project_id AND f.user_dni =  u.user_dni
  CREATE (u)-[:WORKED_ON]->(p)
  """
  session.run(query_1_rel)

  # Para filtrar: {project_id:'PRDLR21531BRET'}   o   {user_dni:'14305313A'}
  query_1 = """MATCH p=()-[:WORKED_ON]->() RETURN p;"""
  session.run(query_1)

  # ---------------------------------------------------------------------------------------------

  # Se puede cambiar Pr_CancerTypes por Projects y asi obtendríamos más información
  query_2_rel = """
  MATCH (p:Fus_Pr_CT)
  MATCH (c:Fus_CT {cancer_code: p.cancer_code})
  CREATE (p)-[:TREATS]->(c)
  """
  session.run(query_2_rel)

  query_2_1 = """
  MATCH p=()-[:TREATS]->({cancer_code: "['melanoma']"}) RETURN p;"""
  session.run(query_2_1)

  query_2_2 = """
  MATCH (p:Fus_Pr_CT)
  SET p.research_type_code = apoc.convert.fromJsonList(p.research_type_code)
  """
  session.run(query_2_2)

  query_2_3 = """
  MATCH (r:Fus_RT)
  SET r.research_type_code = [r.research_type_code]
  """
  session.run(query_2_3)

  query_3_1 = """
  MATCH (r:Fus_RT), (p:Fus_Pr_CT)
  WHERE ANY(item IN r.research_type_code WHERE item IN p.research_type_code)
  RETURN r, p LIMIT 10;
  """
  session.run(query_3_1)

  query_3_2 = """
  MATCH (r:Fus_RT), (p:Fus_Pr_CT)
  WHERE ANY(item IN r.research_type_code WHERE item IN p.research_type_code)
  MERGE (p)-[:RELATED_TO]->(r)
  """
  session.run(query_3_2)

  query_4 = """
  MATCH p1=(p:Fus_Pr_CT)-[:TREATS]->(c:Fus_CT)
  OPTIONAL MATCH p2=(p)-[:RELATED_TO]->(r:Fus_RT)
  RETURN p1, p2 ;
  """
  session.run(query_4)
  # ---------------------------------------------------------------------------------------------
